In [ ]:
# ALL THE IMPORTS NECESSARY
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from geopy.distance import great_circle as vc 
import math as Math

# from mpl_toolkits.basemap import Basemap
# import matplotlib.cm as cm

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import math, time
from keras.models import model_from_json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/LandfallAAAISubmission/"
filePath = path+"IMDDataSSTLandfall.csv"
alldata = pd.read_csv(filePath, index_col=0)
data = alldata.drop(['BOO','date','month','year','time','season', 'grade','oci','diam','cino','pd'], axis = 1)

data.dropna(subset=['lati', 'long'])
data = data.interpolate(method ='linear', limit_direction ='forward') 
data = data.dropna()

keys = list(enumerate(pd.unique(data['cyclone_id'])))
total_hurricane_count = len(pd.unique(data['cyclone_id']))
y = np.zeros((total_hurricane_count))
for x in range(0,total_hurricane_count):
    y[x] = len(data[data['cyclone_id'] == keys[x][1]])

# Now contains how many time instances of data (or rows) each hurricane contains 
hurricane_amount = pd.DataFrame(y)
print(total_hurricane_count)
hurricane_amount.describe()

352


,0
count,352.000000
mean,25.818182
std,16.231969
min,1.000000
25%,13.000000
50%,23.000000
75%,35.000000
max,88.000000


In [ ]:
data.shape

(9088, 6)

In [ ]:
total_hurricane_count

352

In [ ]:
# Print the hurricane key with the amount of data they contain
print ('Top 6 Hurricanes (in terms of data quantity)')
for x in hurricane_amount.nlargest(10, 0).index:
    print (keys[x][1], "-", hurricane_amount.loc[x][0])
    

Top 6 Hurricanes (in terms of data quantity)
2018-13 - 88.0
2003-1 - 79.0
2019-7 - 78.0
2018-11 - 71.0
1996-9 - 70.0
2019-2 - 68.0
2019-8 - 68.0
2005-3 - 65.0
1984-2 - 62.0
1987-1 - 61.0


In [ ]:
# Total amount of data tuples
total_data_count = len(data)
print(total_data_count)
data.head()

9088


,lati,long,cino,ecp,mssw,pd,sst,cyclone_id
36,14.50,91.5,1.50,990.00,25.00,3.10,29.050000,1982-2
37,16.75,91.4,1.75,989.00,27.50,3.78,29.109999,1982-2
38,19.00,91.3,2.00,988.00,30.00,4.46,29.300000,1982-2
39,18.00,91.2,2.17,986.67,33.33,5.62,28.789999,1982-2
40,17.00,91.1,2.33,985.33,36.67,6.77,29.010000,1982-2


In [ ]:
#Generating the features distance and direction

total_hurricane_count = len(pd.unique(data['cyclone_id']))
total_data_count = data.shape[0]

keys = list(enumerate(pd.unique(data['cyclone_id'])))

y = np.zeros(total_hurricane_count)
data['distance'] = np.zeros(total_data_count)
data['direction'] = np.zeros(total_data_count)

# For all hurricanes
for x in range(0, total_hurricane_count):
    t = pd.DataFrame(data[data['cyclone_id'] == keys[x][1]], columns = data.keys()).reset_index(drop = False)
    dst = 0
    prev = (0,0)
    # For all latitude and longitude points of hurricane, calculate the angle of travel and distance
    for p in zip(t['lati'], t['long']):
        if prev == (0,0):
            prev = p
            continue 
        # Stores the distance into the DataFrame
        data.at[t[(t['lati'] == p[0]) & (t['long'] == p[1])]['index'].values[0], 'distance'] =  vc(prev,p).miles
        
        dLon = p[1] - prev[1];  
        temp = float(p[0]) # p[0] is a str?
        y_x = Math.sin(dLon) * Math.cos(temp);
        
        x_x = Math.cos(p[1]) * Math.sin(temp) - Math.sin(p[1]) * Math.cos(temp) * Math.cos(dLon);
        brng = Math.degrees(Math.atan2(y_x, x_x)) 
        if (brng < 0):
            brng+= 360;
        
        # Stores the angle of travel into the DataFrame
        data.at[t[(t['lati'] == p[0]) & (t['long'] == p[1])]['index'].values[0], 'direction']= brng
        dst += vc(prev,p).miles
        prev = p
    y[x] = dst


In [ ]:
data.describe()

,lati,long,ecp,mssw,sst,distance,direction
count,9088.000000,9088.000000,9088.000000,9088.000000,9088.000000,9088.000000,9088.000000
mean,16.574570,79.474877,993.536972,37.358440,28.569236,23.188506,123.984305
std,5.235065,10.856014,19.596431,20.325478,1.121417,28.444491,122.084723
min,1.500000,42.900000,25.000000,2.000000,23.240000,0.000000,-0.000000
25%,12.500000,71.000000,990.236726,25.000000,27.849998,0.000000,0.000000
50%,16.500000,83.500000,998.000000,30.000000,28.606043,18.781851,140.978625
75%,21.000000,87.500000,1000.599162,40.000000,29.320000,34.387664,188.865992
max,40.800000,103.000000,1010.000000,140.000000,32.010000,653.593495,359.974016


In [ ]:
pip install global-land-mask

     |████████████████████████████████| 1.8MB 4.6MB/s 


In [ ]:
# Finding the data between cyclone origination on sea and its landfall
from global_land_mask import globe
# bm = Basemap()   # default: projection='cyl'
latiIndex = 0
longIndex = 1
dataTillLand = []
for key in np.unique(data['cyclone_id']):
  data1 = data[data.loc[:, 'cyclone_id'] == key].loc[:,:].values
  length = len(data1)
  #print("key ========  ",key, '   length   ', length)
  checks = []
  for i in range(length):
    data2 = data1[i]
    lati = data2[latiIndex]
    long = data2[longIndex]
    isLand = globe.is_land(lati, long)
    checks.append(isLand)
  # print('checks  ', checks)
  smallestOceanFromAbove = -1
  smallestLandFromBelow = -1
  for i in range(length):
    check = checks[i]
    if not check:
      smallestOceanFromAbove = i
      break
  for i in range(length-1, -1,-1):
    check = checks[i]
    if check:
      smallestLandFromBelow = i
      j = i-1
      while(j>=0):
        check = checks[j]
        # print('j   ', j, '   check  ', check, '    smallestLandFromBelow   ',smallestLandFromBelow)
        if check:
          smallestLandFromBelow = j
        else:
          break
        j = j-1
      break
  # print('smallestOceanFromAbove  ',smallestOceanFromAbove)
  # print("smallestLandFromBelow  ",smallestLandFromBelow)
  if smallestOceanFromAbove !=-1 and smallestLandFromBelow!=-1:
    for i in range(smallestOceanFromAbove, smallestLandFromBelow+1, 1):
      dataTillLand.append(data1[i])

dataToLandfall = pd.DataFrame(dataTillLand, columns = data.columns)
print(dataToLandfall.shape)
dataToLandfall.to_csv(path+'IMDDataTillLandfall.csv')

(3988, 8)


In [ ]:
print(len(pd.unique(dataToLandfall['cyclone_id'])))

206
